<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/%EB%B0%95%EA%B1%B4%EC%9A%B0__%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selenium 국회의원 데이터 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromum-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromum-browser/chromedriver')

     |████████████████████████████████| 911kB 2.7MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [901 kB]
Get:14 http://

## 라이브러리 import

In [3]:
import os
import shutil
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.request import urlretrieve

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 국회의원 기본정보 스크래핑

In [9]:
def get_politician_info(dir_name, wd):

  profile_info = wd.find_element_by_class_name('depAnalProfileTbl')

  cols = profile_info.find_elements_by_tag_name("th")
  columns = [col.text for col in cols]

  infos = profile_info.find_elements_by_tag_name("td")
  detail_info = [info.text for info in infos]

  info_dic = { col:info for col, info in zip(columns, detail_info) }

  return info_dic


# 국회의원 뉴스 인용문

In [22]:
def get_news_quote(dir_name, name, wd, news_max= 5):
  
  page_no = 0
  news_count = 0
  news_total = wd.find_element_by_id('newsInQoutTotalCount').text[1:-2]
  news_file = open(dir_name + '/' + name + ' 뉴스 인용문.txt', 'w')

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('newsInQuotListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f"뉴스 인용문 {page_no} 페이지")

      box_list = wd.find_element_by_id('newsInQuotList')
      quotes = box_list.find_elements_by_tag_name('b')
      for quote in quotes:
        news_file.write(quote.text + '\n')
        news_count += 1
        print(f"{news_count}/{news_total}", [quote.text])
      
        if news_count >= news_max:
          break

      if news_count >= news_max:
          break

      next_button = pagination.find_elements_by_tag_name('li')[-1]
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:  
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  news_file.close()

## 국회의원 회의록

In [34]:
def get_record_quote(dir_name, name, wd, record_max = 5):

  page_no = 0
  record_count = 0
  record_total = wd.find_element_by_id('recordTotalCount').text
  record_file = open(dir_name + '/' + name + ' 회의록 인용문.txt', 'w')

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('recordTableListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f"회의록 인용문 {page_no} 페이지")

      record_table = wd.find_element_by_id('recordTableList')
      record_list = record_table.find_elements_by_tag_name('a')
      for record in record_list:
        record.click()
        time.sleep(1)

        quote_list = wd.find_element_by_id('birefTableList')
        quotes = quote_list.find_elements_by_tag_name('td')
        for quote in quotes:
          record_file.write(quote.text + '\n')

        record_count += 1
        print(f"{record_count}/{record_total}", [quote.text])
      
        if record_count >= record_max:
          break

      if record_count >= record_max:
          break

      next_button = pagination.find_elements_by_tag_name('li')[-1]
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:  
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  record_file.close()

## `scraping()`: 스크래핑

In [28]:
def scraping(dir_name, politician_max = 3 ):
  page_no = 0
  politician_count = 0
  politician_df = pd.DataFrame()

  wd = webdriver.Chrome('chromedriver', options = chrome_options)
  wd.execute_script('window.open("about:blank", "_blank");')
  tabs = wd.window_handles

  while True:

    try:
      page_no += 1
      wd.switch_to.window(tabs[0])

      url = f'https://www.bigkinds.or.kr/v2/depthAnalysis/assembly.do?page={page_no}'
      wd.get(url)
      politician_total = wd.find_element_by_xpath('//*[@id="contents"]/div/div[3]/div/button[1]/span').text[1:-1]
      politician_items = wd.find_elements_by_class_name("assembly-item")

      if not politician_items:
        break

      for item in politician_items:
        wd.switch_to.window(tabs[0])

        politician_count += 1
        print("------------------------------------------------")
        print(f"국회의원 {politician_count}/{politician_total} 명")
        name = item.find_element_by_class_name('assembly-item__profile__name').text
        name = name.split('\n')[0]
        print(name)

        detail_link = item.find_element_by_tag_name('a').get_attribute('href')

        wd.switch_to.window(tabs[1])
        wd.get(detail_link)

        print("[프로필 이미지 다운로드]")
        profile_image = wd.find_element_by_class_name('depAnalProfileImg')

        image_src = profile_image.find_element_by_tag_name('img').get_attribute('src')
        file_name = dir_name + '/' + name + '.jpg'
        print(file_name)
        urlretrieve(image_src, file_name)

        print("[기본 정보 스크래핑]")
        politician_dic = get_politician_info(dir_name, wd)
        print("  ",politician_dic)
        politician_df = politician_df.append(politician_dic, ignore_index= True)

        print("[뉴스 인용문 스크래핑]")
        wd.find_element_by_id('depthAnalTab2_1').find_element_by_tag_name('a').click()
        get_news_quote(dir_name, name, wd, 4)

        print("[회의록 인용문 스크래핑]")
        get_record_quote(dir_name, name, wd)

        if politician_count >= politician_max:
          break

      if politician_count >= politician_max:
          break
    except AttributeError as e:
      print(e)
      break

    except NoSuchElementException as e:
      print(e)
      break  
    
  wd.close()

  return politician_df


# 스크래핑 시작

In [35]:
dir_name = './politician'
if os.path.isdir(dir_name):
  shutil.rmtree(dir_name)

os.makedirs(dir_name)
print(f'{dir_name} 디렉토리 생성')

politician_df = scraping(dir_name)

./politician 디렉토리 생성
------------------------------------------------
국회의원 1/300 명
강기윤 (姜起潤)
[프로필 이미지 다운로드]
./politician/강기윤 (姜起潤).jpg
[기본 정보 스크래핑]
   {'이름': '강기윤(姜起潤)', '이름(영문)': 'KANG GIYUN', '출생일': '1960-06-04', '정당': '국민의힘', '지역구': '경남 창원시성산구', '소속위원회': '보건복지위원회', '당선기록': '재선(19대, 21대)', '사무실전화': '02-784-1751', '홈페이지': 'http://blog.naver.com/ggotop', '이메일': 'ggotop@naver.com', '보좌관': '강종길 , 김홍광', '경력': '[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사 창원대학교 대학원 행정학 박사 [경력] 보건복지위원회 위원 미래통합당 소상공인살리기 특별위원회 부위원장 미래통합당 경남도당 민생특위 위원장 제19대 국회의원 (새누리당/경남 창원시 성산구) 새누리당 원내부대표'}
[뉴스 인용문 스크래핑]
뉴스 인용문 1 페이지
1/6 ['자가격리 중인 분들은 힘들고 불편하시겠지만 철저한 방역을 위해 조금만 더 외부 활동하시는 것을 참아주시길 부탁드린다']
2/6 ['문재인 정부가 공공의대 설립을 추진하면서 관련 법안이 국회 보건복지위원회의 심사조차 이뤄지지 않았는데도, 공공의대 설립 지역을 ‘전북 남원’으로 특정하여 내년도 정부예산안에 2억 3000만원을 반영시켰다 공공의대법안은 국회 통과는 둘째 치고 복지위 법안소위에서 심의조차 되지 않은 상황인데 어떻게 보건복지부가 법안 통과를 전제로 기재부 협의까지 마치고 국..']
3/6 ['공공의대법안은 국회 통과는 둘째 치고 복지위 법안소위에서 심의조차 되지 않은 상황인데 어떻게 보건복지부가 법안 통과를 전제로 기재부 협의까지 마치고 국회에 예산안을 제출할 수 있는지 의문 복지부

## 스크래핑 확인

In [12]:
politician_df

,경력,당선기록,보좌관,사무실전화,소속위원회,이름,이름(영문),이메일,정당,지역구,출생일,홈페이지
0,[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사...,"재선(19대, 21대)","강종길 , 김홍광",02-784-1751,보건복지위원회,강기윤(姜起潤),KANG GIYUN,ggotop@naver.com,국민의힘,경남 창원시성산구,1960-06-04,http://blog.naver.com/ggotop
1,,초선(21대),"박홍규 , 정운태",,국방위원회,강대식(姜大植),KANG DAESIK,,국민의힘,대구 동구을,1959-11-02,
2,2018. 10. ~ 2019. 5. 민주연구원 자치발전연구센터 본부장 2016. ...,초선(21대),"서용선 , 유진우",02-784-2747~9,교육위원회,강득구(姜得求),KANG DEUKGU,mainsail440@daum.net,더불어민주당,경기 안양시만안구,1963-05-27,https://blog.naver.com/dulipapa
3,,초선(21대),"강민승 , 박진우",,정무위원회,강민국(姜旻局),KANG MINKUK,,국민의힘,경남 진주시을,1971-03-03,
4,전) 서울시교육청 혁신학교운영위원회 위원장 전) 교육부 민주시민교육자문위원회 위원 ...,초선(21대),"손성조 , 전경원",02-784-2477,교육위원회,강민정(姜旼姃),KANG MINJUNG,kmj2020edu@gmail.com,열린민주당,비례대표,1961-04-26,https://blog.naver.com/kmgedu21


In [36]:
!ls politician/

'강대식 (姜大植).jpg'		     '강득구 (姜得求) 회의록 인용문.txt'
'강대식 (姜大植) 뉴스 인용문.txt'    '강기윤 (姜起潤).jpg'
'강대식 (姜大植) 회의록 인용문.txt'  '강기윤 (姜起潤) 뉴스 인용문.txt'
'강득구 (姜得求).jpg'		     '강기윤 (姜起潤) 회의록 인용문.txt'
'강득구 (姜得求) 뉴스 인용문.txt'


In [37]:
from google.colab import files
files.download('./politician/강대식 (姜大植).jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
!cat './politician/강대식 (姜大植) 회의록 인용문.txt'

해당 인용문이 존재하지 않습니다.
해당 인용문이 존재하지 않습니다.

그렇지 않지요? 그런데 7월 6일 자에 보면 정세현 민주평통자문회의 수석부의장께서 김어준의 뉴스공장에 출연해 가지고 9․19 군사합의 이후 실시한 한미연합훈련에 대해서 군사합의 위반이라고 이렇게 주장을 했는데 여기에 대해서 장관께서 어떻게 생각하십니까?

허위사실에 대해서 이야기를 할 때에는 국방부에서의 명확한 답변을 내놔야 된다고 저는 그렇게 보고 있거든요. 우리 국민들이 봤을 때 호도되지 않고 정말 정확한 팩트 있는 그런 사실들을 우리 국민이 알 수 있게끔 이렇게 국방부에서 해 줘야 된다고 생각하는데 장관님 어떻게 생각하세요?

수석부의장은 대통령의 어떤 명을 받아서 이렇게 하는 아주 중요한 위치에 계시는 분이 이런 허위사실에 대해서…… (발언시간 초과로 마이크 중단) 1분만 더 주십시오.

또 한 가지 물어보겠습니다. 한미연합훈련이 9․19 합의에 위반됩니까?

예, 됐습니다. 됐습니다.

예, 무슨 말씀인지 알겠습니다. 이게 꼭히 통합신공항이 510만 대구․경북민들의 염원이 담긴 사업이다 이렇게는 제가 말씀을안 드리겠습니다. 왜냐하면 생각하기도 싫지만이 사업이 만약에 좌초된다고 봤을 때는 대구․경북을 넘어서서 예측하기 힘든 정치적 갈등의소용돌이 속에 빠질 수 있다 이런 생각을 한번해 보고요. 주무부처로서 책임감을 가지고 며칠 남지는 않았지만 그동안 수십 년간 소음 피해로 인해서 또고도 제한으로 인해서 재산권 침해로 인해서 고통받은 대구시민들의 희생을 헤아려서 마지막까지 군위군을 설득하는 데 최선을 다해 줄 수 있겠습니까?

그런데 아직까지 군위군은 그럴 마음이 전연 없고요. 언론에 보면 장관님이 내일 일정으로 군위군수하고 면담이 있는 걸로 알고 있는데 혹시 설득을 할 수 있는 대안을 갖고 계십니까?

장관님, 조금 전에 질의했던 내용과 연결하겠습니다. 군위군이 진짜로 유치하면 제일 클리어합니다, 그렇지요?

뭐 조금만이라도 그런……

방위력개선비 중 약 한 9200억 원이 삭감된 이후